In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import ParameterSampler, KFold

import warnings
warnings.filterwarnings("ignore")

import torch

def check_gpu_status():
    if torch.cuda.is_available():
        print(f"可用GPU数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"显存使用情况: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB 已用, {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB 总分配")
            print(f"显存空余: {torch.cuda.memory_reserved(i) - torch.cuda.memory_allocated(i):.2f} 字节\n")
    else:
        print("未检测到 GPU！")

check_gpu_status()

可用GPU数量: 2
GPU 0: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节

GPU 1: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节



In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

sample = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True) 

In [4]:
def date(Df):

    Df['Policy Start Date'] = pd.to_datetime(Df['Policy Start Date'])
    Df['Year'] = Df['Policy Start Date'].dt.year
    Df['Day'] = Df['Policy Start Date'].dt.day
    Df['Month'] = Df['Policy Start Date'].dt.month
    Df['Month_name'] = Df['Policy Start Date'].dt.month_name()
    Df['Day_of_week'] = Df['Policy Start Date'].dt.day_name()
    Df['Week'] = Df['Policy Start Date'].dt.isocalendar().week
    Df['Year_sin'] = np.sin(2 * np.pi * Df['Year'])
    Df['Year_cos'] = np.cos(2 * np.pi * Df['Year'])
    Df['Month_sin'] = np.sin(2 * np.pi * Df['Month'] / 12) 
    Df['Month_cos'] = np.cos(2 * np.pi * Df['Month'] / 12)
    Df['Day_sin'] = np.sin(2 * np.pi * Df['Day'] / 31)  
    Df['Day_cos'] = np.cos(2 * np.pi * Df['Day'] / 31)
    Df['Group']=(Df['Year']-2020)*48+Df['Month']*4+Df['Day']//7
    
    Df.drop('Policy Start Date', axis=1, inplace=True)

    return Df
train = date(train)
test = date(test)

cat_cols = [col for col in train.columns if train[col].dtype == 'object']
feature_cols = list(test.columns)
class CategoricalEncoder:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def frequency_encode(self, cat_cols, feature_cols, drop_org=False):
        combined = pd.concat([self.train, self.test], axis=0, ignore_index=True)

        new_cat_cols = [] 
        for col in cat_cols:
            freq_encoding = combined[col].value_counts().to_dict()
            
            self.train[f"{col}_freq"] = self.train[col].map(freq_encoding).astype('float')
            self.test[f"{col}_freq"] = self.test[col].map(freq_encoding).astype('float')

            new_col_name = f"{col}_freq"
            new_cat_cols.append(new_col_name)
            feature_cols.append(new_col_name)
            if drop_org:
                feature_cols.remove(col)

        return self.train, self.test, new_cat_cols, feature_cols
encoder = CategoricalEncoder(train, test)
train, test, cat_cols, feature_cols = encoder.frequency_encode(cat_cols, feature_cols, drop_org=True)

train = train[feature_cols + ['Premium Amount']]
test = test[feature_cols]

In [5]:
X = train.drop('Premium Amount', axis=1)  
y = train['Premium Amount']

y_log = np.log1p(y)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [6]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

# 定义 RMSLE 评估函数
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# 定义 Optuna 的目标函数
def objective(trial, X, y, n_splits=5, random_state=42):
    # 定义超参数搜索空间
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000, step=1000),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.03, 0.05]),
        'max_depth': trial.suggest_int('max_depth', 4, 10, step=2),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0, step=0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'gamma': trial.suggest_float('gamma', 0, 0.3, step=0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 0.1, step=0.01),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0, step=0.1),
    }

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof = np.zeros(len(X))
    fold_scores = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold + 1}")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

        model = xgb.XGBRegressor(
            **param,
            objective='reg:squarederror',
            random_state=random_state,
            eval_metric="rmse",
            verbosity=0,
            tree_method='gpu_hist',  # 使用 GPU 加速
            booster='gbtree',
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=300,
            verbose=False,
        )

        # 预测和评估
        oof[valid_idx] = np.maximum(0, model.predict(X_valid))
        fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
        fold_scores.append(fold_rmsle)

        print(f"    Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")

    mean_score = np.mean(fold_scores)
    print(f"Mean RMSLE: {mean_score:.4f}")
    
    return mean_score

# 使用 Optuna 进行超参数优化
def optimize_hyperparameters(X, y, n_trials=10, n_splits=5):
    study = optuna.create_study(direction='minimize')  # 目标是最小化 RMSLE
    study.optimize(lambda trial: objective(trial, X, y, n_splits), n_trials=n_trials)
    
    print("\nBest Parameters:", study.best_params)
    print(f"Best RMSLE: {study.best_value:.4f}")
    return study.best_params, study.best_value, study

# 执行超参数优化
best_params, best_rmsle, study = optimize_hyperparameters(X, y)

# 使用最佳参数训练最终模型
kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = []
for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
    print(f"Final Model Training - Fold {fold + 1}")
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

    model = xgb.XGBRegressor(
        **best_params,
        objective='reg:squarederror',
        random_state=42,
        eval_metric="rmse",
        tree_method='gpu_hist',
        booster='gbtree',
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=300,
        verbose=False,
    )
    models.append(model)

# 对测试集进行预测
test_predictions = np.zeros(len(test))
for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)

# 将预测结果保存到 CSV 文件
sample['Premium Amount'] = test_predictions
sample.to_csv('submission_xgb_optuna.csv', index=False)

[I 2024-12-05 06:05:45,727] A new study created in memory with name: no-name-c2fcb635-69a5-4fdf-939e-5285e0da8ab1


  Fold 1
    Fold 1 RMSLE: 1.0466
  Fold 2
    Fold 2 RMSLE: 1.0457
  Fold 3
    Fold 3 RMSLE: 1.0467
  Fold 4
    Fold 4 RMSLE: 1.0447
  Fold 5


[I 2024-12-05 06:06:36,103] Trial 0 finished with value: 1.0459571931124476 and parameters: {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'gamma': 0.3, 'reg_alpha': 0.0, 'reg_lambda': 1.0}. Best is trial 0 with value: 1.0459571931124476.


    Fold 5 RMSLE: 1.0462
Mean RMSLE: 1.0460
  Fold 1
    Fold 1 RMSLE: 1.0484
  Fold 2
    Fold 2 RMSLE: 1.0472
  Fold 3
    Fold 3 RMSLE: 1.0480
  Fold 4
    Fold 4 RMSLE: 1.0463
  Fold 5


[I 2024-12-05 06:08:34,799] Trial 1 finished with value: 1.0475079773818377 and parameters: {'n_estimators': 2000, 'learning_rate': 0.03, 'max_depth': 4, 'subsample': 0.7, 'colsample_bytree': 1.0, 'min_child_weight': 2, 'gamma': 0.2, 'reg_alpha': 0.07, 'reg_lambda': 0.0}. Best is trial 0 with value: 1.0459571931124476.


...

    Fold 5 RMSLE: 1.0464
Mean RMSLE: 1.0461
  Fold 1
    Fold 1 RMSLE: 1.0465
  Fold 2
    Fold 2 RMSLE: 1.0456
  Fold 3
    Fold 3 RMSLE: 1.0466
  Fold 4
    Fold 4 RMSLE: 1.0446
  Fold 5


[I 2024-12-05 06:17:50,942] Trial 8 finished with value: 1.045891285939501 and parameters: {'n_estimators': 2000, 'learning_rate': 0.01, 'max_depth': 6, 'subsample': 0.7999999999999999, 'colsample_bytree': 0.8999999999999999, 'min_child_weight': 3, 'gamma': 0.2, 'reg_alpha': 0.0, 'reg_lambda': 0.6000000000000001}. Best is trial 4 with value: 1.045793759983385.


    Fold 5 RMSLE: 1.0461
Mean RMSLE: 1.0459
  Fold 1
    Fold 1 RMSLE: 1.0501
  Fold 2
    Fold 2 RMSLE: 1.0488
  Fold 3
    Fold 3 RMSLE: 1.0496
  Fold 4
    Fold 4 RMSLE: 1.0481
  Fold 5


[I 2024-12-05 06:18:55,462] Trial 9 finished with value: 1.0492030550143006 and parameters: {'n_estimators': 1000, 'learning_rate': 0.01, 'max_depth': 4, 'subsample': 1.0, 'colsample_bytree': 0.7999999999999999, 'min_child_weight': 5, 'gamma': 0.2, 'reg_alpha': 0.09, 'reg_lambda': 0.8}. Best is trial 4 with value: 1.045793759983385.


    Fold 5 RMSLE: 1.0494
Mean RMSLE: 1.0492

Best Parameters: {'n_estimators': 3000, 'learning_rate': 0.05, 'max_depth': 8, 'subsample': 0.7, 'colsample_bytree': 0.8999999999999999, 'min_child_weight': 3, 'gamma': 0.1, 'reg_alpha': 0.07, 'reg_lambda': 0.1}
Best RMSLE: 1.0458
Final Model Training - Fold 1
Final Model Training - Fold 2
Final Model Training - Fold 3
Final Model Training - Fold 4
Final Model Training - Fold 5


In [7]:
import optuna
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

# 定义 RMSLE 评估函数
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# 定义目标函数
def objective(trial, X, y, n_splits=1, random_state=42):
    # 定义超参数搜索空间
    param = {
        'iterations': trial.suggest_int('iterations', 2000, 4000, step=1000),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.03, 0.05]),
        'depth': trial.suggest_int('depth', 4, 10, step=2),
        'l2_leaf_reg': trial.suggest_categorical('l2_leaf_reg', [0.5, 0.7, 1.0, 2.0]),
        'bagging_temperature': trial.suggest_categorical('bagging_temperature', [0.5, 1.0, 1.5]),
        'random_strength': trial.suggest_int('random_strength', 1, 5),
    }

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof = np.zeros(len(X))
    fold_scores = []

    for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold + 1}")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

        model = CatBoostRegressor(
            **param,
            random_seed=random_state,
            eval_metric="RMSE",
            verbose=0,
            task_type='GPU',
        )
        
        model.fit(
            X_train, y_train,
            eval_set=(X_valid, y_valid),
            early_stopping_rounds=300,
        )

        # 预测和评估
        oof[valid_idx] = np.maximum(0, model.predict(X_valid))
        fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
        fold_scores.append(fold_rmsle)

        print(f"    Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")

    mean_score = np.mean(fold_scores)
    print(f"Mean RMSLE: {mean_score:.4f}")
    
    return mean_score

# 使用 Optuna 进行超参数优化
def optimize_hyperparameters(X, y, n_trials=10, n_splits=5):
    study = optuna.create_study(direction='minimize')  # 目标是最小化 RMSLE
    study.optimize(lambda trial: objective(trial, X, y, n_splits), n_trials=n_trials)
    
    print("\nBest Parameters:", study.best_params)
    print(f"Best RMSLE: {study.best_value:.4f}")
    return study.best_params, study.best_value

# 执行超参数优化
best_params, best_rmsle = optimize_hyperparameters(X, y)

# 使用最佳参数训练最终模型
final_model = CatBoostRegressor(
    **best_params,
    random_seed=42,
    eval_metric="RMSE",
    verbose=0,
    task_type='GPU',
)

# 在整个训练集上训练最终模型
final_model.fit(X, np.log1p(y))

# 对测试集进行预测
test_predictions = np.maximum(0, np.expm1(final_model.predict(test)))  # 预测并还原数据

# 将预测结果保存到 CSV 文件
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')  # 替换为你的样本文件路径
sample_submission['Premium Amount'] = test_predictions

# 保存预测结果
sample_submission.to_csv('submission_optuna_catboost.csv', index=False)

[I 2024-12-05 06:19:53,131] A new study created in memory with name: no-name-f7ff8fe6-ad6d-481d-93ae-70a083ecea34


  Fold 1
    Fold 1 RMSLE: 1.0484
  Fold 2
    Fold 2 RMSLE: 1.0476
  Fold 3
    Fold 3 RMSLE: 1.0483
  Fold 4
    Fold 4 RMSLE: 1.0465
  Fold 5


[I 2024-12-05 06:21:44,889] Trial 0 finished with value: 1.0477973528039441 and parameters: {'iterations': 4000, 'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 2.0, 'bagging_temperature': 1.0, 'random_strength': 5}. Best is trial 0 with value: 1.0477973528039441.


    Fold 5 RMSLE: 1.0481
Mean RMSLE: 1.0478
  Fold 1
    Fold 1 RMSLE: 1.0487
  Fold 2
    Fold 2 RMSLE: 1.0478
  Fold 3
    Fold 3 RMSLE: 1.0486
  Fold 4
    Fold 4 RMSLE: 1.0467
  Fold 5


[I 2024-12-05 06:23:53,202] Trial 1 finished with value: 1.04801378883259 and parameters: {'iterations': 3000, 'learning_rate': 0.03, 'depth': 6, 'l2_leaf_reg': 2.0, 'bagging_temperature': 1.0, 'random_strength': 5}. Best is trial 0 with value: 1.0477973528039441.


...

    Fold 5 RMSLE: 1.0481
Mean RMSLE: 1.0478
  Fold 1
    Fold 1 RMSLE: 1.0485
  Fold 2
    Fold 2 RMSLE: 1.0477
  Fold 3
    Fold 3 RMSLE: 1.0486
  Fold 4
    Fold 4 RMSLE: 1.0464
  Fold 5


[I 2024-12-05 06:37:02,892] Trial 8 finished with value: 1.047863843684337 and parameters: {'iterations': 2000, 'learning_rate': 0.05, 'depth': 10, 'l2_leaf_reg': 1.0, 'bagging_temperature': 1.0, 'random_strength': 3}. Best is trial 5 with value: 1.0473814579387084.


    Fold 5 RMSLE: 1.0481
Mean RMSLE: 1.0479
  Fold 1
    Fold 1 RMSLE: 1.0491
  Fold 2
    Fold 2 RMSLE: 1.0483
  Fold 3
    Fold 3 RMSLE: 1.0491
  Fold 4
    Fold 4 RMSLE: 1.0472
  Fold 5


[I 2024-12-05 06:38:24,361] Trial 9 finished with value: 1.0485086607961898 and parameters: {'iterations': 2000, 'learning_rate': 0.05, 'depth': 6, 'l2_leaf_reg': 1.0, 'bagging_temperature': 1.5, 'random_strength': 3}. Best is trial 5 with value: 1.0473814579387084.


    Fold 5 RMSLE: 1.0488
Mean RMSLE: 1.0485

Best Parameters: {'iterations': 2000, 'learning_rate': 0.03, 'depth': 8, 'l2_leaf_reg': 0.5, 'bagging_temperature': 0.5, 'random_strength': 5}
Best RMSLE: 1.0474


In [8]:
import pandas as pd

# 加载两个已有的 CSV 文件
sample_submission_xgb = pd.read_csv('submission_xgb_optuna.csv')
sample_submission_catboost = pd.read_csv('submission_optuna_catboost.csv')

# 提取预测结果列 'Premium Amount'
xgb_preds = sample_submission_xgb['Premium Amount']
catboost_preds = sample_submission_catboost['Premium Amount']

# 设置加权系数（例如 1:1 权重）
w1, w2 = 0.6, 0.4

# 加权融合预测值
final_preds = (w1 * xgb_preds + w2 * catboost_preds) / (w1 + w2)

# 将融合后的结果存入 DataFrame 中
sample_submission_xgb['Premium Amount'] = final_preds

# 保存加权融合后的结果为新的 CSV 文件
sample_submission_xgb.to_csv('submission_fused6_4.csv', index=False)

# 显示前几行结果确认
print(sample_submission_xgb.head())


        id  Premium Amount
0  1200000      793.580108
1  1200001      798.630011
2  1200002      781.575901
3  1200003      788.266926
4  1200004      746.247652


In [9]:
import pandas as pd

# 加载两个已有的 CSV 文件
sample_submission_xgb = pd.read_csv('submission_xgb_optuna.csv')
sample_submission_catboost = pd.read_csv('submission_optuna_catboost.csv')

# 提取预测结果列 'Premium Amount'
xgb_preds = sample_submission_xgb['Premium Amount']
catboost_preds = sample_submission_catboost['Premium Amount']

# 设置加权系数（例如 1:1 权重）
w1, w2 = 0.7, 0.3

# 加权融合预测值
final_preds = (w1 * xgb_preds + w2 * catboost_preds) / (w1 + w2)

# 将融合后的结果存入 DataFrame 中
sample_submission_xgb['Premium Amount'] = final_preds

# 保存加权融合后的结果为新的 CSV 文件
sample_submission_xgb.to_csv('submission_fused7_3.csv', index=False)

# 显示前几行结果确认
print(sample_submission_xgb.head())


        id  Premium Amount
0  1200000      793.082134
1  1200001      797.522141
2  1200002      784.460864
3  1200003      790.074581
4  1200004      748.147668


In [10]:
import pandas as pd

# 加载两个已有的 CSV 文件
sample_submission_xgb = pd.read_csv('submission_xgb_optuna.csv')
sample_submission_catboost = pd.read_csv('submission_optuna_catboost.csv')

# 提取预测结果列 'Premium Amount'
xgb_preds = sample_submission_xgb['Premium Amount']
catboost_preds = sample_submission_catboost['Premium Amount']

# 设置加权系数（例如 1:1 权重）
w1, w2 = 0.8, 0.2

# 加权融合预测值
final_preds = (w1 * xgb_preds + w2 * catboost_preds) / (w1 + w2)

# 将融合后的结果存入 DataFrame 中
sample_submission_xgb['Premium Amount'] = final_preds

# 保存加权融合后的结果为新的 CSV 文件
sample_submission_xgb.to_csv('submission_fused8_2.csv', index=False)

# 显示前几行结果确认
print(sample_submission_xgb.head())


        id  Premium Amount
0  1200000      792.584159
1  1200001      796.414271
2  1200002      787.345827
3  1200003      791.882235
4  1200004      750.047685


In [11]:
import pandas as pd

# 加载两个已有的 CSV 文件
sample_submission_xgb = pd.read_csv('submission_xgb_optuna.csv')
sample_submission_catboost = pd.read_csv('submission_optuna_catboost.csv')

# 提取预测结果列 'Premium Amount'
xgb_preds = sample_submission_xgb['Premium Amount']
catboost_preds = sample_submission_catboost['Premium Amount']

# 设置加权系数（例如 1:1 权重）
w1, w2 = 0.9, 0.1

# 加权融合预测值
final_preds = (w1 * xgb_preds + w2 * catboost_preds) / (w1 + w2)

# 将融合后的结果存入 DataFrame 中
sample_submission_xgb['Premium Amount'] = final_preds

# 保存加权融合后的结果为新的 CSV 文件
sample_submission_xgb.to_csv('submission_fused9_1.csv', index=False)

# 显示前几行结果确认
print(sample_submission_xgb.head())


        id  Premium Amount
0  1200000      792.086185
1  1200001      795.306402
2  1200002      790.230790
3  1200003      793.689889
4  1200004      751.947701


In [12]:
import pandas as pd

# 加载两个已有的 CSV 文件
sample_submission_xgb = pd.read_csv('submission_xgb_optuna.csv')
sample_submission_catboost = pd.read_csv('submission_optuna_catboost.csv')

# 提取预测结果列 'Premium Amount'
xgb_preds = sample_submission_xgb['Premium Amount']
catboost_preds = sample_submission_catboost['Premium Amount']

# 设置加权系数（例如 1:1 权重）
w1, w2 = 0.95, 0.05

# 加权融合预测值
final_preds = (w1 * xgb_preds + w2 * catboost_preds) / (w1 + w2)

# 将融合后的结果存入 DataFrame 中
sample_submission_xgb['Premium Amount'] = final_preds

# 保存加权融合后的结果为新的 CSV 文件
sample_submission_xgb.to_csv('submission_fused95.csv', index=False)

# 显示前几行结果确认
print(sample_submission_xgb.head())


        id  Premium Amount
0  1200000      791.837198
1  1200001      794.752467
2  1200002      791.673272
3  1200003      794.593716
4  1200004      752.897709
